In [3]:
from dotenv import load_dotenv
import os

model_name=os.environ.get("model_name")
base_url=os.environ.get("base_url")
api_key=os.environ.get("api_key")
print(model_name)
print(base_url)
print(api_key)

inclusionAI/Ling-mini-2.0
https://api.siliconflow.cn
sk-iujgwjycqgmvzxycfgyynuowipaykmbbcneerzvdnehpqvfs


In [5]:
import os
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv


load_dotenv() 

BASE_URL = "https://api.siliconflow.cn/v1"
API_KEY = "sk-iujgwjycqgmvzxycfgyynuowipaykmbbcneerzvdnehpqvfs"
EMBEDDING_MODEL = "Qwen/Qwen3-Embedding-0.6B"

# --- 测试函数 ---
def test_custom_embedding():
    """
    使用自定义 base_url 和模型测试 OpenAIEmbeddings。
    """
    test_texts = [
        "你好，世界！",
        "Hello, World!",
        "什么是向量嵌入？"
    ]
    
    print("--- 正在初始化嵌入模型 ---")
    print(f"Base URL: {BASE_URL}")
    print(f"Model: {EMBEDDING_MODEL}")
    
    try:
        # 使用 OpenAIEmbeddings 初始化嵌入模型，并指定自定义参数
        # ⚠️ 注意：api_key 和 openai_api_base 是关键
        embeddings = OpenAIEmbeddings(
            model=EMBEDDING_MODEL,
            openai_api_base=BASE_URL,
            openai_api_key=API_KEY
            # 其他参数（如 chunk_size）如果需要也可以在这里设置
        )
        
        print("\n--- 正在请求嵌入向量 ---")
        
        # 1. 测试单句嵌入
        query = test_texts[0]
        embedding_vector = embeddings.embed_query(query)
        print(f"✅ 成功嵌入查询 '{query}'")
        print(f"向量维度: {len(embedding_vector)}")
        # print(f"部分向量值: {embedding_vector[:5]}...") # 可选：打印部分向量
        
        # 2. 测试多句嵌入 (Documents)
        document_embeddings = embeddings.embed_documents(test_texts)
        print(f"\n✅ 成功嵌入 {len(document_embeddings)} 个文档")
        
        # 检查所有返回的向量是否维度一致
        first_dim = len(document_embeddings[0])
        all_match = all(len(vec) == first_dim for vec in document_embeddings)
        print(f"所有向量维度是否一致: {all_match} (维度: {first_dim})")
        
        print("\n🌟 嵌入模型测试成功！")
        
    except Exception as e:
        # 如果出现 404 Not Found 或其他 API 错误，将在这里捕获
        print(f"\n❌ 嵌入模型测试失败！请检查配置和服务：")
        print(f"错误类型: {type(e).__name__}")
        print(f"错误信息: {e}")
        print("\n💡 检查建议：1. 确认 base_url 和 api_key 正确； 2. 确认模型名称 'Qwen/Qwen3-Embedding-0.6B' 在该服务上可用。")


if __name__ == "__main__":
    test_custom_embedding()

--- 正在初始化嵌入模型 ---
Base URL: https://api.siliconflow.cn/v1
Model: Qwen/Qwen3-Embedding-0.6B

--- 正在请求嵌入向量 ---
✅ 成功嵌入查询 '你好，世界！'
向量维度: 1024

✅ 成功嵌入 3 个文档
所有向量维度是否一致: True (维度: 1024)

🌟 嵌入模型测试成功！
